
# Preprocessing phase 1:
##    - Splitting hours from an original file
##    - Save its data on seperate files

In [25]:
import gc
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from IPython.display import display

# conda install openpyxl

In [26]:
root_path = os.getcwd()
original_dataset_path = os.path.join(root_path, 'datasets', 'Original_MCP_Data')
preprocessed_dataset_path = os.path.join(root_path, 'datasets', 'Preprocess_Phase_1')

os.makedirs(name=preprocessed_dataset_path, exist_ok=True)

In [27]:
def get_need_preprocessing_paths(load_path):
    # Create an empty list to store path of files that need preprocessing
    
    need_preprocessing = []

    for year in os.listdir(path=load_path):
        for month in os.listdir(path=os.path.join(load_path, year)):
            month_path = os.path.join(load_path, year, month)
            
            need_preprocessing += [os.path.join(month_path, day) for day in os.listdir(path=month_path)]

    return need_preprocessing

In [28]:
def correct_time(x):
    day = x[:2]
    month = x[3:5]
    year = x[6:10]
    hour = x[11:13]
    minute = x[14:16]
    second = x[17:19]
    return f'{year}-{month}-{day} {hour}:{minute}:{second}'

In [29]:
def process_one_hour_data(one_hour_original_data, time_column, invalid_column_names, save_path):

    # Create a new empty dictionary for storing one hour final data
    one_hour_final_data = {}

    # Check if the first row of the first column of the one hour original data is 'Bid curve chart data (Reference time)'
    # In another word, check if the first column contain column names
    assert one_hour_original_data.iloc[0, 0] == time_column
    
    ################################################
    # Add name, value to empty dictionary
    #===============================================
    # For each row
    for row_index in range(len(one_hour_original_data)):
        name, value = one_hour_original_data.iloc[row_index, 0], one_hour_original_data.iloc[row_index, 1]

        # Check if the name is not in the invalid column names
        if name not in invalid_column_names:

            # Creating column names
            if name not in one_hour_final_data:
                one_hour_final_data[name] = []

            # Fixing 'Bid curve chart data (Reference time)'
            if name == time_column:
                value = value.replace(' +', '')

                assert len(value) == 19, f'Time should have 19 characters, but got {len(value)}'

                # Change time format
                value = correct_time(x=value)  # %Y-%m-%d %H:%M:%S
                # Determine save path
                save_path = os.path.join(save_path, value[0:4], value[5:7], value[8:10], value[11:13])
                # Original save name
                original_save_name = value.replace('-', '_').replace(' ', '_').replace(':', '_')

            # Add this value to the empty dictionary
            one_hour_final_data[name].append(value)
    ################################################

    ################################################
    # All name, value in one_hour_final_data
    # should have the same length
    #===============================================
    # Get maximum number of values in one_hour_final_data
    max_count_values = max([len(v) for v in one_hour_final_data.values()])

    # Checking length of names and values in one_hour_final_data
    for k, v in one_hour_final_data.items():

        # Number of values in 'Price value' and 'Volume value' should be equal to max_count_values
        if len(v) != 1:
            assert len(v) == max_count_values, f'{k} should have {max_count_values} values, but got {len(v)}'

        # Other names should have only 1 value. It will be broadcast max_count_values times
        else:
            one_hour_final_data[k] = [v[0] for _ in range(max_count_values)]
    ################################################


    # Create one_hour_final_data dataframe
    one_hour_final_data = pd.DataFrame(
        data=one_hour_final_data,
        columns=one_hour_final_data.keys()
    )
    one_hour_final_data[time_column] = pd.to_datetime(arg=one_hour_final_data[time_column], format='%Y-%m-%d %H:%M:%S')

    # Create directories for storing this new prepared data if they are not exist
    os.makedirs(name=save_path, exist_ok=True)

    ################################################
    # Check if the save name already exists
    # (for different data with exactly the same date)
    #===============================================
    # Saving file
    if len(original_save_name) != 19:
        raise ValueError(f'Invalid tmp name {original_save_name}!')
    else:
        duplicate_count = 0
        save_name = f'{original_save_name}_{duplicate_count}.csv'
        while os.path.exists(path=os.path.join(save_path, save_name)):
            duplicate_count += 1
            save_name = f'{original_save_name}_{duplicate_count}.csv'

    one_hour_final_data.to_csv(
        path_or_buf=os.path.join(save_path, save_name),
        columns=one_hour_final_data.columns,
        index=True
    )
    ################################################
    
    # Delete this file in RAM for efficiency
    del one_hour_final_data

    # Clear the RAM
    gc.collect()

In [30]:
def process_one_file(file_path, time_column, invalid_column_names, save_path):

    # Read the original file
    original_dataset = pd.read_excel(io=file_path, header=None)

    # Number of columns should be even
    assert len(original_dataset.columns) % 2 == 0, f'Error in {file_path} file:\n' \
        f'Number of columns should be even, but got {len(original_dataset.columns)}'

    # For every even number (column index) in range of number of original dataset columns
    for i in range(0, len(original_dataset.columns), 2):

        # process every two column (one hour data) and then save that in separate files
        process_one_hour_data(
            one_hour_original_data=original_dataset.iloc[:, i:i+2],
            time_column=time_column,
            invalid_column_names=invalid_column_names,
            save_path=save_path,
        )

In [31]:
def preprocess_phase_one(data_path, save_path, invalid_column_names, time_column):

    # Get file paths that need preprocessing
    need_preprocessing_paths = get_need_preprocessing_paths(load_path=data_path)

    # For displaying the process as progressive bar
    t = tqdm(need_preprocessing_paths)

    # for each file that needs preprocessing
    for path in t:
        
        t.set_description_str(desc=f'Preprocessing {path}')

        # Process the file
        process_one_file(
            file_path=path,
            time_column=time_column,
            invalid_column_names=invalid_column_names,
            save_path=save_path
        )

In [32]:
preprocess_phase_one(
    data_path=original_dataset_path,
    save_path=os.path.join(preprocessed_dataset_path, '1_phase_one'),
    invalid_column_names=[float('nan'), None, np.nan],
    time_column='Bid curve chart data (Reference time)'
)

Preprocessing /home/naeim_md93/Projects/NordPool_MCP_Forecasting/datasets/Original_MCP_Data/2019/11_November_2019/mcp_data_report_11-11-2019-00_00_00.xls:   0%|          | 0/3636 [00:00<?, ?it/s]

,0,1
0,Bid curve chart data (Reference time),11.11.2019 00:00:00
1,Bid curve chart data (Currency for the values),EUR
2,Bid curve chart data (Reference price object ID),SP1
3,Bid curve chart data (Volume for accepted bloc...,400.9
4,Bid curve chart data (Volume for accepted bloc...,1356
...,...,...
3908,NaN,NaN
3909,NaN,NaN
3910,NaN,NaN
3911,NaN,NaN


,2,3
0,Bid curve chart data (Reference time),11.11.2019 01:00:00
1,Bid curve chart data (Currency for the values),EUR
2,Bid curve chart data (Reference price object ID),SP1
3,Bid curve chart data (Volume for accepted bloc...,400.9
4,Bid curve chart data (Volume for accepted bloc...,1356
...,...,...
3908,NaN,NaN
3909,NaN,NaN
3910,NaN,NaN
3911,NaN,NaN


,4,5
0,Bid curve chart data (Reference time),11.11.2019 02:00:00
1,Bid curve chart data (Currency for the values),EUR
2,Bid curve chart data (Reference price object ID),SP1
3,Bid curve chart data (Volume for accepted bloc...,400.9
4,Bid curve chart data (Volume for accepted bloc...,1356
...,...,...
3908,NaN,NaN
3909,NaN,NaN
3910,NaN,NaN
3911,NaN,NaN


,6,7
0,Bid curve chart data (Reference time),11.11.2019 03:00:00
1,Bid curve chart data (Currency for the values),EUR
2,Bid curve chart data (Reference price object ID),SP1
3,Bid curve chart data (Volume for accepted bloc...,400.9
4,Bid curve chart data (Volume for accepted bloc...,1356
...,...,...
3908,NaN,NaN
3909,NaN,NaN
3910,NaN,NaN
3911,NaN,NaN


...

,42,43
0,Bid curve chart data (Reference time),11.11.2019 21:00:00
1,Bid curve chart data (Currency for the values),EUR
2,Bid curve chart data (Reference price object ID),SP1
3,Bid curve chart data (Volume for accepted bloc...,665.6
4,Bid curve chart data (Volume for accepted bloc...,1430
...,...,...
3908,Volume value,59862.420646
3909,Price value,2999.946
3910,Volume value,59954.462633
3911,Price value,3000


,44,45
0,Bid curve chart data (Reference time),11.11.2019 22:00:00
1,Bid curve chart data (Currency for the values),EUR
2,Bid curve chart data (Reference price object ID),SP1
3,Bid curve chart data (Volume for accepted bloc...,665.6
4,Bid curve chart data (Volume for accepted bloc...,1342.6
...,...,...
3908,NaN,NaN
3909,NaN,NaN
3910,NaN,NaN
3911,NaN,NaN


,46,47
0,Bid curve chart data (Reference time),11.11.2019 23:00:00
1,Bid curve chart data (Currency for the values),EUR
2,Bid curve chart data (Reference price object ID),SP1
3,Bid curve chart data (Volume for accepted bloc...,528.6
4,Bid curve chart data (Volume for accepted bloc...,1303.3
...,...,...
3908,Volume value,59992.494965
3909,NaN,NaN
3910,NaN,NaN
3911,NaN,NaN


Preprocessing /home/naeim_md93/Projects/NordPool_MCP_Forecasting/datasets/Original_MCP_Data/2019/11_November_2019/mcp_data_report_11-11-2019-00_00_00.xls:   0%|          | 0/3636 [00:07<?, ?it/s]


NameError: name 'xxx' is not defined